# Tratamento de Valores Ausentes com Imputação

São vários os métodos de imputação usados para substituir valores ausentes em conjuntos de dados por valores estimados. Eles ajudam a melhorar a qualidade dos dados, permitindo que eles sejam usados em análises estatísticas ou no treinamento de modelos de aprendizado de máquina, que muitas vezes não lidam bem com valores ausentes.

Por que usar imputação?

* Evita a perda de informações que ocorre ao excluir linhas ou colunas com valores ausentes.
* Mantém a representatividade do conjunto de dados.
* Melhora a performance e a precisão de modelos, que podem ser prejudicados pela ausência de dados.

## Configurando dados

In [1]:
# Importando biblioteca pandas
import pandas as pd

# Importando biblioteca numpy
import numpy as np

# Importando a função minmax_scale da biblioteca scikit-learn
from sklearn.preprocessing import minmax_scale

# Função para medir acurácia
def err(df, df_):
    # Calculando o MAE
    mae = (df - df_).abs().mean().mean()
    # Calculando MSE
    mse = ((df - df_) ** 2).mean().mean()
    # Calculando Correlação
    correlation = df.corrwith(df_).mean()
    # Exibindo resultados
    print(f'\nAcurácia para os dados usados:\n\nMAE: {mae:.4f}\nMSE: {mse:.4f}\nCor: {correlation:.4f}')

# Lendo arquivo CSV e transformando em um DataFrame
df = pd.read_csv('datas/diabetes.csv')

# Varrendo colunas do DataFrame
for column in df:

    # Dimensiona valores para mesma escala (entre 0 e 1) para fins de visualização de gráfico
    df[column] = minmax_scale(df[column])

# Excluindo valores nulos existentes
df = df.dropna()

# Declarando novo dataframe que terão dados nulos forjados
df_nan = df.copy()

# Selecionando índices aleatórios para substituir por NaN
missing_indices = np.random.choice(
    df_nan.size,  # Número total de elementos no DataFrame
    int(df_nan.size * 0.1),  # Número de elementos que serão substituídos por NaN (10%)
    replace=False
)

# Substituindo os valores nos índices escolhidos por NaN
df_nan_1d = df_nan.values.flatten()  # Transforma o DataFrame em um array 1D
df_nan_1d[missing_indices] = np.nan  # Insere NaN nos índices selecionados
df_nan[:] = df_nan_1d.reshape(df_nan.shape)  # Reshape para o formato original

# Exibindo contagem de valores nulos por coluna
print("\nNulos por coluna:\n")
print(df_nan.isnull().sum())


Nulos por coluna:

preg    80
plas    76
pres    71
skin    70
insu    76
mass    94
pedi    70
age     71
dtype: int64


## Substituição por constante:

* Preenche valores ausentes com um valor fixo, como 0 ou "Desconhecido" (em variáveis categóricas).

* Útil quando um valor específico tem um significado claro.

In [2]:
# Declarando novo dataframe que terão dados nulos transformados em constante
df_cons = df_nan.copy()

# Substituindo os valores NaN por 0
df_cons = df_cons.fillna(0)

err(df, df_cons)


Acurácia para os dados usados:

MAE: 0.0322
MSE: 0.0157
Cor: 0.7975


## Substituição pela média/mediana/moda:

* Para variáveis numéricas, usa:
    * Média: mean
    * Mediana: median

* Para variáveis categóricas, usa:
    * Moda (valor mais frequente).

* Simples de implementar, mas pode distorcer a variabilidade dos dados.

In [3]:
from sklearn.impute import SimpleImputer

# Declarando novo dataframe que terão dados nulos transformados em media
df_mean = df_nan.copy()

for column in df_mean.columns:

    # Definindo estratégia de imputação
    imputer = SimpleImputer(strategy='mean')  # Estratégia: 'mean', 'median', 'most_frequent'

    # Aplicando imputação nos dados faltantes
    df_mean[column] = imputer.fit_transform(df_mean[[column]])

err(df, df_mean)


Acurácia para os dados usados:

MAE: 0.0116
MSE: 0.0022
Cor: 0.9561


## Substituição usando o K-Nearest Neighbors (KNN):

* Substitui valores ausentes com base na média/mediana/moda de k-vizinhos mais próximos.

* Captura melhor as relações entre variáveis.

In [4]:
from sklearn.impute import KNNImputer

# Declarando novo dataframe que terá os dados nulos transformados
df_knn = df_nan.copy()

# Definindo imputação para k vizinhos (5)
imputer = KNNImputer(n_neighbors=5)

# Aplicando imputação no DataFrame inteiro, pois o imputador espera uma matriz bidimensional
df_knn.iloc[:, :] = imputer.fit_transform(df_knn)

err(df, df_knn)


Acurácia para os dados usados:

MAE: 0.0099
MSE: 0.0019
Cor: 0.9624


## Outros métodos de substituição

* Preenchimento pelo último ou próximo valor conhecido (forward/backward fill):

    * Preenche com o último valor não ausente encontrado (ou o próximo, no caso do backward fill).
    * Comum em séries temporais.

* Regressão:

    * Usa regressão para prever valores ausentes com base em outras variáveis.
    * Exemplo: Prever valores ausentes de altura com base em idade e peso.

* Interpolação:

    * Estima valores ausentes com base em técnicas matemáticas, como Linear, Polinomial, Splines.

* Modelos de Aprendizado de Máquina:

    * Métodos como árvores de decisão ou redes neurais podem imputar valores com maior precisão ao capturar relações complexas nos dados.

* Múltipla Imputação:

    * Gera várias estimativas para os valores ausentes e combina os resultados para refletir a incerteza da imputação.

Cada método possui suas próprias especificidades e custo computacional. A efetividade de uso de cada um se dará a partir do tipo de dado que está sendo tratado, não havendo um que se destaque sobre outro para toda e qualquer situação.